In [19]:
import csv
import sys

from util import Node, StackFrontier, QueueFrontier

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [20]:
directory = "C:/Users/STW/Documents/Harvard EdX CS50/Search/Projects/Degrees/small"

def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(directory+"/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(directory+"/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(directory+"/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [30]:
print(load_data(directory))
print(names)
print(people)
print(movies)


None
{'kevin bacon': {'102'}, 'tom cruise': {'129'}, 'cary elwes': {'144'}, 'tom hanks': {'158'}, 'mandy patinkin': {'1597'}, 'dustin hoffman': {'163'}, 'chris sarandon': {'1697'}, 'demi moore': {'193'}, 'jack nicholson': {'197'}, 'bill paxton': {'200'}, 'sally field': {'398'}, 'valeria golino': {'420'}, 'gerald r. molen': {'596520'}, 'gary sinise': {'641'}, 'robin wright': {'705'}, 'emma watson': {'914612'}}
{'102': {'name': 'Kevin Bacon', 'birth': '1958', 'movies': {'112384', '104257'}}, '129': {'name': 'Tom Cruise', 'birth': '1962', 'movies': {'95953', '104257'}}, '144': {'name': 'Cary Elwes', 'birth': '1962', 'movies': {'93779'}}, '158': {'name': 'Tom Hanks', 'birth': '1956', 'movies': {'112384', '109830'}}, '1597': {'name': 'Mandy Patinkin', 'birth': '1952', 'movies': {'93779'}}, '163': {'name': 'Dustin Hoffman', 'birth': '1937', 'movies': {'95953'}}, '1697': {'name': 'Chris Sarandon', 'birth': '1942', 'movies': {'93779'}}, '193': {'name': 'Demi Moore', 'birth': '1962', 'movies': 

In [22]:
def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


In [23]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """
    # Initialize frontier to the source
    start = Node(state=self.source, parent=None, action=None)
    frontier = QueueFrontier()
    frontier.add(start)
    
    # Initialize an empty explored set
    self.explored = set()

    # Keep looping until solution found
    while True:

        # If nothing left in frontier, then no path
        if frontier.empty():
            raise Exception("no solution")

        # Choose a node from the frontier
        node = frontier.remove()
        self.num_explored += 1

        # If node is the goal, then we have a solution
        # starting from the goal node (last):
        if node.state == self.goal:
            # make a list for the path (relations)
            path = []
#             actions = []
#             cells = []
            # if you haven't re-traced the path all the way to the beginning
            while node.parent is not None:
                # add the current node's attributes (movie, person_id) to the path list
                path.append((node.action,node.state))
#                 actions.append(node.action)
#                 cells.append(node.state)
                # then set the node "one up" (one closer to the source node)
                node = node.parent
            path.reverse()
#             actions.reverse()
#             cells.reverse()
            self.solution = path
            return

        # Mark node as explored
        self.explored.add(node)

        # Add neighbors to frontier
        for action, state in self.neighbors_for_person(node.state):
            if not frontier.contains_state(state) and state not in self.explored:
                child = Node(state=state, parent=node, action=action)
                frontier.add(child)

    
    # TODO
    # raise NotImplementedError
    

In [24]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]
    

In [25]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors



In [26]:
if __name__ == "__main__":
    main()

SystemExit: Usage: python degrees.py [directory]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
